In [1]:
#Distance of Croos and 45Degree
import pandas as pd

# Define the path to your .txt file
file_path = 'C:/Users/rjhyu/파이썬 연습/23.12.09 오류 원인 찾기/C15mm00deg.txt'

# Read the file and extract the x, y, z values
data = []
with open(file_path, 'r') as file:
    for line in file:
        values = line.strip().split()  # Split the line into individual values
        if len(values) >= 3:
            try:
                x, y, z = float(values[0]), float(values[1]), float(values[2])  # Extract x, y, z
                data.append([x, y, z])
            except ValueError:
                # Skip lines that cannot be converted to floats
                continue

# Create a DataFrame from the extracted data
df = pd.DataFrame(data, columns=['x', 'y', 'z'])

# Calculate the median x value and define the range for analysis (10cm up and down from the median)
median_x = df['x'].median()
range_start = median_x - 0.1
range_end = median_x + 0.1

# First filter the DataFrame for the x value range around the median
filtered_df_x = df[(df['x'] >= range_start) & (df['x'] <= range_end)]

# Within this filtered data, further filter based on the specified y value ranges
filtered_df = filtered_df_x[((filtered_df_x['y'] >= 0) & (filtered_df_x['y'] <= 0.03)) | ((filtered_df_x['y'] >= 0.07) & (filtered_df_x['y'] <= 0.1))]

# Calculate the average z value from the doubly filtered DataFrame
average_z_value = filtered_df['z'].mean()

# Define the reference point using the calculated average z value
reference_point = (0.05, average_z_value)
print(reference_point)

# Find the minimum and maximum x values to divide the sections accurately
min_x = df['x'].min()
max_x = df['x'].max()
section_width = (max_x - min_x) / 20

# Initialize the results DataFrame
results_df = pd.DataFrame(columns=['Section', 'Y Distance (mm)', 'Left Point Y', 'Left Point Z', 'Right Point Y', 'Right Point Z'])

# Process the sections
for i in range(20):
    section_min_x = min_x + (i * section_width)
    section_max_x = min_x + ((i + 1) * section_width)
    section_label = f"{section_min_x:.2f} to {section_max_x:.2f}"
    
    # Filter rows based on the x value section and z value margin
    filtered_df = df[(df['x'] >= section_min_x) & (df['x'] < section_max_x) & (abs(df['z'] - reference_point[1]) <= 0.001)]
    
    # Sort filtered_df based on the y value
    filtered_df = filtered_df.sort_values(by='y')
    
    # Find the closest point to the left of y=0.05
    left_point = filtered_df[filtered_df['y'] < reference_point[0]].iloc[-1] if not filtered_df[filtered_df['y'] < reference_point[0]].empty else None
    
    # Find the closest point to the right of y=0.05
    right_point = filtered_df[filtered_df['y'] > reference_point[0]].iloc[0] if not filtered_df[filtered_df['y'] > reference_point[0]].empty else None
    
    # Calculate the y distance between the two points and append the result to results_df
    if left_point is not None and right_point is not None:
        distance_y = abs(right_point['y'] - left_point['y']) * 1000  # Convert to mm
        new_row = pd.DataFrame({
            'Section': [section_label],
            'Y Distance (mm)': [distance_y],
            'Left Point Y': [left_point['y']],
            'Left Point Z': [left_point['z']],
            'Right Point Y': [right_point['y']],
            'Right Point Z': [right_point['z']]
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)
    else:
        new_row = pd.DataFrame({
            'Section': [section_label],
            'Y Distance (mm)': ['N/A'],
            'Left Point Y': ['N/A'],
            'Left Point Z': ['N/A'],
            'Right Point Y': ['N/A'],
            'Right Point Z': ['N/A']
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)

# Save results_df to a new Excel file
results_df.to_excel('C:/Users/rjhyu/파이썬 연습/23.12.09 오류 원인 찾기/results.xlsx', index=False)


(0.05, 0.023106427464904655)


In [7]:
#Distance of Raidal
import pandas as pd

# Define the path to your .txt file
file_path = 'C:/Users/rjhyu/파이썬 연습/for txt file/row data/230414 Radial 재 측정/230414RERadial_2mm_60deg_all.txt'

# Read the file and extract the x, y, z values
data = []
with open(file_path, 'r') as file:
    for line in file:
        values = line.strip().split()  # Split the line into individual values
        if len(values) >= 3:
            try:
                x, y, z = float(values[0]), float(values[1]), float(values[2])  # Extract x, y, z
                data.append([x, y, z])
            except ValueError:
                # Skip lines that cannot be converted to floats
                continue

# Create a DataFrame from the extracted data
df = pd.DataFrame(data, columns=['x', 'y', 'z'])

# Filter the DataFrame for the specified y value ranges (now x value ranges)
filtered_df = df[((df['x'] >= 0) & (df['x'] <= 0.03)) | ((df['x'] >= 0.07) & (df['x'] <= 1.0))]

# Calculate the average z value from the filtered DataFrame
average_z_value = filtered_df['z'].mean()

# Define the reference point using the calculated average z value
reference_point = (average_z_value, 0.05)

# Find the minimum and maximum y values to divide the sections accurately (now x values)
min_y = df['y'].min()
max_y = df['y'].max()
section_height = (max_y - min_y) / 20

# Initialize the results DataFrame
results_df = pd.DataFrame(columns=['Section', 'X Distance (mm)', 'Lower Point X', 'Lower Point Z', 'Upper Point X', 'Upper Point Z'])

# Process the sections
for i in range(20):
    section_min_y = min_y + (i * section_height)
    section_max_y = min_y + ((i + 1) * section_height)
    section_label = f"{section_min_y:.2f} to {section_max_y:.2f}"
    
    # Filter rows based on the y value section and z value margin
    filtered_df = df[(df['y'] >= section_min_y) & (df['y'] < section_max_y) & (abs(df['z'] - reference_point[0]) <= 0.001)]
    
    # Sort filtered_df based on the x value
    filtered_df = filtered_df.sort_values(by='x')
    
    # Find the closest point below and above x=0.05 (now y=0.05)
    lower_point = filtered_df[filtered_df['x'] < reference_point[1]].iloc[-1] if not filtered_df[filtered_df['x'] < reference_point[1]].empty else None
    upper_point = filtered_df[filtered_df['x'] > reference_point[1]].iloc[0] if not filtered_df[filtered_df['x'] > reference_point[1]].empty else None
    
    # Calculate the x distance between the two points and append the result to results_df
    if lower_point is not None and upper_point is not None:
        distance_x = abs(upper_point['x'] - lower_point['x']) * 1000  # Convert to mm
        new_row = pd.DataFrame({
            'Section': [section_label],
            'X Distance (mm)': [distance_x],
            'Lower Point X': [lower_point['x']],
            'Lower Point Z': [lower_point['z']],
            'Upper Point X': [upper_point['x']],
            'Upper Point Z': [upper_point['z']]
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)
    else:
        new_row = pd.DataFrame({
            'Section': [section_label],
            'X Distance (mm)': ['N/A'],
            'Lower Point X': ['N/A'],
            'Lower Point Z': ['N/A'],
            'Upper Point X': ['N/A'],
            'Upper Point Z': ['N/A']
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)

# Save results_df to a new Excel file
results_df.to_excel('C:/Users/rjhyu/파이썬 연습/new method/results.xlsx', index=False)
